## Simulator (fleet of uavs)

This notebook serves to get familiar with the simulator

In [1]:
from config import Color
from helpers import clean
from helpers.coordinates import ENUPose, GRAPose
from plan import Plan
from simulator import (
    QGC,
    ConfigGazebo,
    ConfigNovis,
    ConfigQGC,
    Gazebo,
    NoVisualizer,
    Simulator,
)

clean()

## Simulation Configuration

We can easily extend the configuration from Notebook 2 to multiple UAVs. 
We are also adding here a mission delay in secons.

In [2]:
gra_origin = GRAPose(lat=-35.3633280, lon=149.1652241,alt=0,heading=90) 
enu_origin = ENUPose(x=0, y=0, z=gra_origin.alt, heading=gra_origin.heading) 

base_homes= ENUPose.list([  # east, north, up, heading
    (0., 15., 0., 0.),
    (15., 0., 0., 0),
    (5., -20., 0., 30.),
    (-15., -15., 0., 0),
    (-15., 0., 0., 45),

])
base_paths = [Plan.create_square_path(side_len=10, alt=5,heading=0) for _ in base_homes]

colors=[
    Color.BLUE,
    Color.GREEN,
    Color.BLACK,
    Color.ORANGE,
    Color.RED,
]
msn_delays=[0,1,2,3,4] # in seconds

## Visualizer

Extending the visualizer configuration to multiple UAVs is also straightforward.

## Gazebo

In [3]:
gaz_config = ConfigGazebo(origin = enu_origin,
                          world_path="simulator/gazebo/worlds/runway.world")

for path,home,c in zip(base_paths,base_homes,colors):
    gaz_config.add(base_path=path,base_home=home,color=c)
gaz_config.show()

gaz= Gazebo(gaz_config,gra_origin)

## QGroundControl

In [5]:
qgc_config = ConfigQGC(origin = gra_origin)

for path,home,color,delay in zip(base_paths,base_homes,colors,msn_delays):
    qgc_config.add(base_path=path,base_home=home,color=color,mission_delay=delay)
qgc_config.show()

qgc=QGC(qgc_config)

## No Visualizer

In [6]:
novis_config = ConfigNovis(origin=gra_origin)
for home in base_homes:
    novis_config.add(base_home=home)
    
novis=NoVisualizer(novis_config)

## Launch Simulation

**Here we configure the simulation:**
* **visualizers**: Select the desired visualizer for the simulation (only one visualizer at a time is currently supported).    
* **missions**: List of missions for each UAV. Although advanced, customized missions can be built using the `Mission` class defined in the QGroundControl configuration script, here we use the QGroundControl configuration builder for simplicity.  
* **terminals**: Processes to display in the terminal. The available processes are defined in the `sim.py` script: `["launcher", "veh", "logic", "proxy", "gcs"]`.  
* **suppress_output**: Processes whose output should be suppressed. The available processes are defined in the `sim.py` script: `["launcher", "veh", "logic", "proxy", "gcs"]`.  
* **verbose**: Verbosity level (0 = no information, 1 = normal information, 2 = include debugging information).  
* **other arguments**: More advanced options that allow further customization.  


In [7]:
qgc_config.vehicles

[QGCVehicle(home=GRAPose(lat=np.float64(-35.36332799988731), lon=np.float64(149.1650590523038), alt=np.float64(1.761881245894283e-05), heading=90.0), mission=Mission(traj=[QGCWP(pos=GRA(lat=np.float64(-35.363327999887304), lon=np.float64(149.16505905243307), alt=np.float64(5.000017619074236)), color=blue), QGCWP(pos=GRA(lat=np.float64(-35.36332799968695), lon=np.float64(149.16494902072176), alt=np.float64(5.000048941255045)), color=blue), QGCWP(pos=GRA(lat=np.float64(-35.363237866827895), lon=np.float64(149.1649490210275), alt=np.float64(5.000056806796053)), color=blue), QGCWP(pos=GRA(lat=np.float64(-35.363237867028246), lon=np.float64(149.1650590526165), alt=np.float64(5.000025484076233)), color=blue), QGCWP(pos=GRA(lat=np.float64(-35.363327999887304), lon=np.float64(149.16505905243307), alt=np.float64(5.000017619074236)), color=blue)], delay=0, n_items=7)),
 QGCVehicle(home=GRAPose(lat=np.float64(-35.36319280060458), lon=np.float64(149.1652241), alt=np.float64(1.7697898478799932e-05)

In [8]:
simulator = Simulator(
    gra_origin=gra_origin,
	visualizers=[qgc],
	gcs_system_ids={'multicolor':[1,2,3,4,5]},
	missions=[veh.mission for veh in qgc_config.vehicles],
	terminals=['gcs'],
	verbose=1,
)
orac = simulator.launch()

18:38:43 - Oracle ⚪ - INFO - 🚀 GCS multicolor launched (PID 2885099)
18:38:43 - Oracle ⚪ - INFO - 🗺️  QGroundControl launched for 2D visualization — simulation powered by ArduPilot SITL.


## Oracle checking

In [9]:
orac.run()

18:38:43 - Oracle ⚪ - INFO - 🏁 Starting Oracle with 5 vehicles and 1 GCSs
18:38:43 - Oracle ⚪ - INFO - ✅ All background threads started
18:40:08 - Oracle ⚪ - INFO - Received DONE from UAV 1
18:40:08 - Oracle ⚪ - INFO - Received DONE from UAV 2
18:40:08 - Oracle ⚪ - INFO - Received DONE from UAV 3
18:40:09 - Oracle ⚪ - INFO - Received DONE from UAV 4
18:40:10 - Oracle ⚪ - INFO - Received DONE from GCS multicolor
18:40:10 - Oracle ⚪ - INFO - Received DONE from UAV 5
18:40:10 - Oracle ⚪ - ERROR - Retransmit error for 5: sysid 5 not found in grid
18:40:10 - Oracle ⚪ - INFO - ✅ All GCS threads completed
18:40:10 - Oracle ⚪ - INFO - 🎉 Oracle shutdown complete!
